# Face recogniation, Attendance system and auto creat csv file using date-time

In [1]:
# Importing the necessory libraries
import numpy as np
import cv2
import face_recognition
import os
from datetime import datetime

In [11]:

path = 'images'# creating path for data input

images = []
classNames = []
myList = os.listdir(path)
#print(myList)
for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])
#print(classNames)

In [7]:
# function to finding the encoding of input data images
def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList
encodeListKnown = findEncodings(images)
print('Encodings Complete')

Encodings Complete


In [8]:
# Python script to create an empty file 
# with current date as name. 

# importing datetime module 
import datetime 
import csv
import pandas as pd

# datetime.datetime.now() to get 
# current date as filename. 
filename = datetime.datetime.now() 

# create empty file 
def create_file(): 
# Function creates an empty file 
# %d - date, %B - month, %Y - Year 
    with open(filename.strftime("%d %B %Y")+".csv", "w+") as file:
        writer = csv.writer(file)
        writer.writerow(["Name", "Time"])
         

# Driver Code 
create_file() 


In [9]:
# function to mark attendence of recognised face
def markAttendance(name):
    with open(filename.strftime("%d %B %Y")+".csv",'r+') as f:
        myDataList = f.readlines()
        nameList =[]
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
        if name not in  line:
            now = datetime.datetime.now()
            dt_string = now.strftime("%H:%M:%S")
            f.writelines(f'\n{name},{dt_string}')
            

In [10]:
# starting the local camera
cap = cv2.VideoCapture(0)
 while True:
    success, img = cap.read()
    #img = captureScreen()
    imgS = cv2.resize(img,(0,0),None,0.25,0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

    facesCurFrame = face_recognition.face_locations(imgS)
    encodesCurFrame = face_recognition.face_encodings(imgS,facesCurFrame)

    for encodeFace,faceLoc in zip(encodesCurFrame,facesCurFrame):
        matches = face_recognition.compare_faces(encodeListKnown,encodeFace)
        faceDis = face_recognition.face_distance(encodeListKnown,encodeFace)
        #print(faceDis)
        matchIndex = np.argmin(faceDis)
        if matches[matchIndex]:
            name = classNames[matchIndex].upper()
            #print(name)
            y1,x2,y2,x1 = faceLoc
            y1, x2, y2, x1 = y1*4,x2*4,y2*4,x1*4
            cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)
            cv2.rectangle(img,(x1,y2-35),(x2,y2),(0,255,0),cv2.FILLED)
            cv2.putText(img,name,(x1+6,y2-6),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
            markAttendance(name)
 
    cv2.imshow('Webcam',img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
            
# When everything is done, release the captur using keybutton "q"
cap.release()
cv2.destroyAllWindows()